In [51]:
#Fetches the data for the current Série C

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/brc23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[9]

# Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Write the data to a CSV file
with open('dataBRC23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
    


In [52]:
df = pd.read_csv('dataBRC23raw.csv')
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,02/05/2023,São José RS,0x0,Floresta,62.7 %,21.7 %,15.6 %
1,02/05/2023,Volta Redonda,0x1,Pouso Alegre,55.5 %,23.2 %,21.3 %
2,02/05/2023,Manaus,2x1,Náutico,24.0 %,28.6 %,47.4 %
3,03/05/2023,Paysandu,2x1,Aparecidense,46.8 %,30.4 %,22.8 %
4,03/05/2023,América RN,1x2,Ypiranga RS,52.6 %,23.9 %,23.5 %
...,...,...,...,...,...,...,...
185,26/08/2023,Confiança,1x0,Paysandu,51.5 %,23.9 %,24.6 %
186,26/08/2023,Floresta,1x0,América RN,44.5 %,38.6 %,16.9 %
187,26/08/2023,Aparecidense,2x3,Volta Redonda,29.7 %,26.0 %,44.4 %
188,26/08/2023,Figueirense,0x0,Manaus,68.4 %,24.2 %,7.4 %


In [53]:
unique_values = df['Mandante'].unique()
unique_values_sorted = sorted(unique_values)
print(unique_values_sorted)

['Altos', 'Amazonas', 'América RN', 'Aparecidense', 'Botafogo PB', 'Brusque', 'CSA', 'Confiança', 'Figueirense', 'Floresta', 'Manaus', 'Náutico', 'Operário PR', 'Paysandu', 'Pouso Alegre', 'Remo', 'São Bernardo', 'São José RS', 'Volta Redonda', 'Ypiranga RS']


In [54]:

#Padronizes the names of the teams
df.replace({'América RN': 'América-RN',
            'Botafogo PB': 'Botafogo-PB',
            'Operário PR': 'Operário-PR',
            'São José RS': 'São José-RS',
            'Ypiranga RS': 'Ypiranga-RS',
            }, inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [55]:
# This output cleans and organizes the Data.

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
# Changes dtype to datetime in first dataframe
df['Data'] = df['Data'].str.replace('/', '-')
df['Data'] = pd.to_datetime(df['Data'], format='%d-%m-%Y')
df['Temp'] = df['Data'].dt.year
df = df.sort_values(by='Data')
df['Data'] = df['Data'].dt.strftime('%Y-%m-%d')

# Creates a new column with the for the competition name

df = df.assign(Comp='Série C')

# Separates the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]

# Compares scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
# Create a new column to store the season information in the second data frame

# Organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)



# Sort values by date

# Atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

# Atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
# Corrects any possible issue before saving as new file
df = df.dropna()
df = df.drop(index = 0)
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série C,2023,2023-05-02,Volta Redonda,0,1,Pouso Alegre,VV,0,3,2
2,Série C,2023,2023-05-02,Manaus,2,1,Náutico,VC,3,0,3
3,Série C,2023,2023-05-03,Paysandu,2,1,Aparecidense,VC,3,0,4
4,Série C,2023,2023-05-03,América-RN,1,2,Ypiranga-RS,VV,0,3,5
5,Série C,2023,2023-05-03,Brusque,1,0,Amazonas,VC,3,0,6
...,...,...,...,...,...,...,...,...,...,...,...
182,Série C,2023,2023-08-26,Operário-PR,1,0,Brusque,VC,3,0,186
181,Série C,2023,2023-08-26,Remo,3,1,Altos,VC,3,0,187
180,Série C,2023,2023-08-26,Ypiranga-RS,1,1,Botafogo-PB,E,1,1,188
183,Série C,2023,2023-08-26,Náutico,2,2,São Bernardo,E,1,1,189


In [56]:
#Saves the data into a CSV format
df.to_csv('data/dataBRC23.csv', index=False)
df.to_json('data/dataBRC23.json', orient='records')
df


,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série C,2023,2023-05-02,Volta Redonda,0,1,Pouso Alegre,VV,0,3,2
2,Série C,2023,2023-05-02,Manaus,2,1,Náutico,VC,3,0,3
3,Série C,2023,2023-05-03,Paysandu,2,1,Aparecidense,VC,3,0,4
4,Série C,2023,2023-05-03,América-RN,1,2,Ypiranga-RS,VV,0,3,5
5,Série C,2023,2023-05-03,Brusque,1,0,Amazonas,VC,3,0,6
...,...,...,...,...,...,...,...,...,...,...,...
182,Série C,2023,2023-08-26,Operário-PR,1,0,Brusque,VC,3,0,186
181,Série C,2023,2023-08-26,Remo,3,1,Altos,VC,3,0,187
180,Série C,2023,2023-08-26,Ypiranga-RS,1,1,Botafogo-PB,E,1,1,188
183,Série C,2023,2023-08-26,Náutico,2,2,São Bernardo,E,1,1,189
